In [ ]:
from detectron2.utils.logger import setup_logger
setup_logger()
import cv2
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("train_dataset", {}, "COCOAnnotations/train.json", "MergeDataset/")
register_coco_instances("test_dataset", {}, "COCOAnnotations/test.json", "MergeDataset/")


In [ ]:
sample_metadata = MetadataCatalog.get("train_dataset")
dataset_dicts = DatasetCatalog.get("train_dataset")

In [ ]:
import random
from matplotlib import pyplot as plt


for d in random.sample(dataset_dicts, 1):
    print(d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=sample_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("train_dataset",)
cfg.DATASETS.TEST = ("test_dataset",)   # no metrics implemented for this dataset
# cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 16
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")# initialize from model zoo
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")# initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = 2000   # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 # 3 classes (Person, Helmet, Car)
cfg.MODEL.DEVICE = 'cuda'
cfg.OUTPUT_DIR = 'saved_models/FasterRCNN/'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Test model
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model

# Create COCO evaluator
evaluator = COCOEvaluator("test_dataset", cfg, False, output_dir="./output/")

# Build test loader
val_loader = build_detection_test_loader(cfg, "test_dataset")

# Perform inference and evaluation
evaluation_results = inference_on_dataset(trainer.model, val_loader, evaluator)

# Print mAP (assuming COCO format)
print("mAP:", evaluation_results['bbox'])


In [ ]:
# print("mAP per Category:")
# for category, ap in evaluation_results["bbox"].items():
#   print(f"\t{category}: {ap:.4f}")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("test_dataset",)
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode

sample_metadata = MetadataCatalog.get("test_dataset")
dataset_dicts = DatasetCatalog.get("test_dataset")

for d in random.sample(dataset_dicts, 30):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=sample_metadata, 
                   scale=1, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # cv2_imshow(v.get_image()[:, :, ::-1])
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.engine import DefaultPredictor
import random

# Initialize confusion matrix
num_classes = len(MetadataCatalog.get("test_dataset").thing_classes)
conf_matrix = np.zeros((num_classes, num_classes), dtype=int)

# Perform inference
predictor = DefaultPredictor(cfg)
sample_metadata = MetadataCatalog.get("test_dataset")
dataset_dicts = DatasetCatalog.get("test_dataset")

for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    # Process predictions
    v = Visualizer(im[:, :, ::-1], metadata=sample_metadata, scale=1, instance_mode=ColorMode.IMAGE_BW)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.imshow(v.get_image()[:, :, ::-1])
    plt.show()
    # Update confusion matrix
    for i in range(len(outputs["instances"])):
        pred_cls = outputs["instances"].pred_classes[i].item()
        # Assuming ground truth annotations are available in dataset_dicts
        gt_cls = d["annotations"][i]["category_id"]
        conf_matrix[gt_cls, pred_cls] += 1

# Visualize confusion matrix
plt.figure(figsize=(10, 8))
plt.imshow(conf_matrix, interpolation="nearest", cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()
